In [1]:
import os
import numpy as np
import pandas as pd
%pylab inline
%config InlineBackend.figure_formats = ['retina']

import seaborn as sns
sns.set()

import matplotlib.pyplot as plt

Populating the interactive namespace from numpy and matplotlib


In [1]:
filepath = "C:/Users/user/Desktop/data/Datasets/Human_Activity_Recognition_Using_Smartphones_Data.csv"
data = pd.read_csv(filepath, sep=',')
data.head()

NameError: name 'pd' is not defined

In [3]:
data.shape

(10299, 562)

In [4]:
data.dtypes.value_counts()

float64    561
object       1
dtype: int64

In [5]:
float_columns = (data.dtypes == np.float)

print( (data.loc[:,float_columns].max()==1.0).all() )

print( (data.loc[:,float_columns].min()==-1.0).all() )

<ipython-input-5-7499740ce159>:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  float_columns = (data.dtypes == np.float)


True
True


In [6]:
feature_cols = [x for x in data.columns if x !='Activity']


In [7]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

data['Activity'] = le.fit_transform(data['Activity'])

y = data.Activity

In [8]:
correlations = data[feature_cols].corrwith(y)
correlations.sort_values(inplace=True)
feature_cols = correlations.map(abs).sort_values().iloc[-100:].index
feature_cols

Index(['fBodyGyro-max()-Z', 'fBodyAcc-max()-Z', 'tBodyGyro-min()-Z',
       'tBodyAcc-max()-Z', 'fBodyAccJerk-sma()', 'tBodyGyroMag-max()',
       'tBodyAcc-min()-Y', 'tBodyGyroMag-iqr()', 'tBodyAccJerk-sma()',
       'tBodyAcc-iqr()-X', 'fBodyBodyGyroMag-std()', 'fBodyGyro-std()-X',
       'fBodyAcc-mean()-Z', 'fBodyAccJerk-std()-X', 'tBodyAccJerkMag-sma()',
       'tBodyAccJerkMag-mean()', 'tBodyGyroMag-std()', 'fBodyAccJerk-mad()-X',
       'tBodyAccJerk-min()-X', 'tBodyAccJerk-iqr()-X', 'tBodyGyro-mad()-X',
       'fBodyAccJerk-mean()-X', 'tBodyGyroJerk-entropy()-Y',
       'tBodyAccJerk-std()-X', 'fBodyGyro-mean()-X', 'tBodyGyroMag-mad()',
       'tBodyAccJerk-mad()-X', 'fBodyAccMag-iqr()', 'tBodyGyro-std()-X',
       'fBodyGyro-mad()-X', 'fBodyBodyGyroMag-mad()', 'tGravityAccMag-mad()',
       'tBodyAccMag-mad()', 'fBodyAccMag-sma()', 'fBodyAccMag-mean()',
       'fBodyAccMag-std()', 'tBodyAcc-max()-Y', 'fBodyGyro-sma()',
       'tBodyAccMag-std()', 'tGravityAccMag-std()', 'fBody

In [9]:
x = data[feature_cols]
y.value_counts()

0    1944
2    1906
1    1777
3    1722
5    1544
4    1406
Name: Activity, dtype: int64

In [10]:
from sklearn.model_selection import StratifiedShuffleSplit

strat_shuf_split = StratifiedShuffleSplit(n_splits=2, 
                                          test_size=0.3, 
                                          random_state=420)

train_idx, test_idx = next(strat_shuf_split.split(x, y))

X_train = data.loc[train_idx, feature_cols]
y_train = data.loc[train_idx, 'Activity']

X_test  = data.loc[test_idx, feature_cols]
y_test  = data.loc[test_idx, 'Activity']

In [11]:
print (y_train.value_counts(normalize=True))
print (y_test.value_counts(normalize=True))

0    0.188792
2    0.185046
1    0.172562
3    0.167152
5    0.149951
4    0.136496
Name: Activity, dtype: float64
0    0.188673
2    0.185113
1    0.172492
3    0.167314
5    0.149838
4    0.136570
Name: Activity, dtype: float64


In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Standard logistic regression
lr = LogisticRegression(solver='liblinear').fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)

print (round (f1_score(y_test, y_pred_lr, average='weighted'), 4))
print (round (accuracy_score(y_test, y_pred_lr), 4))

0.8519
0.8521


In [13]:
from sklearn.neighbors import KNeighborsClassifier

max_k = 40
f1_scores = list()
accuracy_rates = list() 

for k in range(1, max_k):
    
    knn = KNeighborsClassifier(n_neighbors=k, weights='distance')
    knn = knn.fit(X_train, y_train)
    
    y_pred_knn = knn.predict(X_test)
    f1 = round(f1_score(y_test, y_pred_knn, average='weighted'), 4)
    f1_scores.append((k, f1))
    accuracy = round(accuracy_score(y_test, y_pred_knn), 4)
    accuracy_rates.append((k, accuracy))
    
scores_results = pd.DataFrame(f1_scores, columns=['K', 'Score'])
accuracy_results = pd.DataFrame(accuracy_rates, columns=['K', 'Accuracy Rate'])

In [14]:
pd.concat([scores_results, accuracy_results], axis=1)

,K,Score,K,Accuracy Rate
0,1,0.8670,1,0.8667
1,2,0.8670,2,0.8667
2,3,0.8724,3,0.8722
3,4,0.8773,4,0.8770
4,5,0.8793,5,0.8793
5,6,0.8879,6,0.8877
6,7,0.8798,7,0.8796
7,8,0.8821,8,0.8822
8,9,0.8834,9,0.8832
9,10,0.8804,10,0.8803


In [15]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=420)
dt = dt.fit(X_train, y_train)

print (dt.tree_.node_count, dt.tree_.max_depth)

from sklearn.model_selection import GridSearchCV

param_grid = {'max_depth':range(1, dt.tree_.max_depth+1, 2),
              'max_features': range(1, len(dt.feature_importances_)+1)}

GR = GridSearchCV(DecisionTreeClassifier(random_state=420),
                  param_grid=param_grid,
                  scoring='accuracy',
                  n_jobs=-1)

GR = GR.fit(X_train, y_train)

print (GR.best_estimator_.tree_.node_count, GR.best_estimator_.tree_.max_depth)

y_pred_dt = GR.predict(X_test)

print (round (f1_score(y_test, y_pred_dt, average='weighted'), 4))
print (round (accuracy_score(y_test, y_pred_dt), 4))

1221 19
871 11
0.8203
0.8204
